In [ ]:
import os
import json
from openai import AzureOpenAI

# Load the environment variables from .env file
from dotenv import load_dotenv

load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

In [ ]:
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,  
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint = AZURE_OPENAI_ENDPOINT
    )

In [3]:
# Create an assistant
assistant = client.beta.assistants.create(
    name="Data Visualization",
    instructions=f"You are a helpful AI assistant who makes interesting visualizations based on data." 
    f"You have access to a sandboxed environment for writing and testing code."
    f"When you are asked to create a visualization you should follow these steps:"
    f"1. Write the code."
    f"2. Anytime you write new code display a preview of the code to show your work."
    f"3. Run the code to confirm that it runs."
    f"4. If the code is successful display the visualization."
    f"5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
    tools=[{"type": "code_interpreter"}],
    model=AZURE_OPENAI_DEPLOYMENT
)

In [4]:
print(assistant.model_dump_json(indent=2))

{
  "id": "asst_N6sXLXuTEzprWtrM3tr0biCf",
  "created_at": 1733837111,
  "description": null,
  "instructions": "You are a helpful AI assistant who makes interesting visualizations based on data.You have access to a sandboxed environment for writing and testing code.When you are asked to create a visualization you should follow these steps:1. Write the code.2. Anytime you write new code display a preview of the code to show your work.3. Run the code to confirm that it runs.4. If the code is successful display the visualization.5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
  "metadata": {},
  "model": "gpt-4o",
  "name": "Data Visualization",
  "object": "assistant",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ],
  "response_format": "auto",
  "temperature": 1.0,
  "tool_resources": {
    "code_interpreter": {
      "file_ids": []
    },
    "file_search": null
  },
  "top_p": 1.0

In [5]:
# Create a thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_lrrFdgnHTqppXvZ3CTV5y4D5', created_at=1733837116, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [6]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a visualization of a sinewave"
)

In [7]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_LnA5Foo9BAXv4VxpphNUYSyO",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Create a visualization of a sinewave"
          },
          "type": "text"
        }
      ],
      "created_at": 1733837120,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_lrrFdgnHTqppXvZ3CTV5y4D5"
    }
  ],
  "object": "list",
  "first_id": "msg_LnA5Foo9BAXv4VxpphNUYSyO",
  "last_id": "msg_LnA5Foo9BAXv4VxpphNUYSyO",
  "has_more": false
}


In [8]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [12]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

completed


In [13]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_aF1lt8Qx1Ve5c9DMBLsOCB4f",
      "assistant_id": "asst_N6sXLXuTEzprWtrM3tr0biCf",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "image_file": {
            "file_id": "assistant-bS0e0WReiP9owUKuZrZOcbSD",
            "detail": null
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here's the visualization of a sinewave. It shows the sine function for \\(x\\) values ranging from 0 to \\(4\\pi\\).\n\nIf you have any specific requirements or would like to see another type of visualization, feel free to let me know!"
          },
          "type": "text"
        }
      ],
      "created_at": 1733837132,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_hXzafGfp9ajBSdeyJxeRvEZk",
      "status": null,
     

In [14]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(image_file_id)

assistant-bS0e0WReiP9owUKuZrZOcbSD


In [15]:
content = client.files.content(image_file_id)

image= content.write_to_file("sinewave.png")

In [16]:
from PIL import Image

# Display the image in the default image viewer
image = Image.open("sinewave.png")
image.show()

In [17]:
# Add a new user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Show me the code you used to generate the sinewave"
)

In [18]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions  but these will override the default instructions
)

In [19]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

completed


In [20]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_OmfJONucZKtkrBIZ1TyWfS1S",
      "assistant_id": "asst_N6sXLXuTEzprWtrM3tr0biCf",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Certainly! Here's the code that was used to generate the sinewave and create the visualization:\n\n```python\nimport numpy as np\nimport matplotlib.pyplot as plt\n\n# Generate x values from 0 to 4*pi\nx = np.linspace(0, 4 * np.pi, 1000)\n# Compute the sine of these x values\ny = np.sin(x)\n\n# Plot the sine wave\nplt.figure(figsize=(10, 6))\nplt.plot(x, y, label='Sine wave')\nplt.title('Sine Wave')\nplt.xlabel('x')\nplt.ylabel('sin(x)')\nplt.legend()\nplt.grid(True)\nplt.show()\n```\n\nThis code:\n1. Imports the necessary libraries: `numpy` and `matplotlib.pyplot`.\n2. Generates 1000 linearly spaced values between 0 and \\(4\\pi\\) for the \\(x\\) axis.\n3. Computes the sine of these \\(x\\) values.\n4. Plots the sin

In [21]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

Certainly! Here's the code that was used to generate the sinewave and create the visualization:

```python
import numpy as np
import matplotlib.pyplot as plt

# Generate x values from 0 to 4*pi
x = np.linspace(0, 4 * np.pi, 1000)
# Compute the sine of these x values
y = np.sin(x)

# Plot the sine wave
plt.figure(figsize=(10, 6))
plt.plot(x, y, label='Sine wave')
plt.title('Sine Wave')
plt.xlabel('x')
plt.ylabel('sin(x)')
plt.legend()
plt.grid(True)
plt.show()
```

This code:
1. Imports the necessary libraries: `numpy` and `matplotlib.pyplot`.
2. Generates 1000 linearly spaced values between 0 and \(4\pi\) for the \(x\) axis.
3. Computes the sine of these \(x\) values.
4. Plots the sine wave using Matplotlib, adds labels, a title, and a legend, and displays the plot.


In [22]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I prefer visualizations in darkmode can you change the colors to make a darkmode version of this visualization."
)

# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [24]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

completed


In [25]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_6R5LAEPfCIRYN4SlsuVTkMNH",
      "assistant_id": "asst_N6sXLXuTEzprWtrM3tr0biCf",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "image_file": {
            "file_id": "assistant-Yyx8v3OMur1b8Lnk8MJ1Bbg2",
            "detail": null
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here's the sinewave visualization in dark mode. The background is now dark, and the sine wave, as well as the labels and grid lines, have been adjusted to light colors for better visibility.\n\nIf you have any further customization requests, please let me know!"
          },
          "type": "text"
        }
      ],
      "created_at": 1733837193,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_hxLhaXOE9xmiCN6RPAQTlFTX",
 

In [26]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
image_file_id = data['data'][0]['content'][0]['image_file']['file_id'] # index numbers can vary if you have had a different conversation over the course of the thread.

print(image_file_id)

assistant-Yyx8v3OMur1b8Lnk8MJ1Bbg2


In [27]:
content = client.files.content(image_file_id)
image= content.write_to_file("dark_sine.png")

# Display the image in the default image viewer
image = Image.open("dark_sine.png")
image.show()